<h1>Reccomendation System</h1>

<h2>Importing Libraries</h2>

In [1]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns
import re
import math

<h2>Importing Database</h2>

In [2]:
apps = pd.read_csv(r"googleplaystore.csv")

<h1>Data Cleanup</h1>

In [3]:
display(apps)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [4]:
apps = apps.drop(columns=['Type', 'Price', 'Category','Last Updated', 'Current Ver'])
apps.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,19M,"10,000+",Everyone,Art & Design,4.0.3 and up
1,Coloring book moana,3.9,967,14M,"500,000+",Everyone,Art & Design;Pretend Play,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,8.7M,"5,000,000+",Everyone,Art & Design,4.0.3 and up
3,Sketch - Draw & Paint,4.5,215644,25M,"50,000,000+",Teen,Art & Design,4.2 and up
4,Pixel Draw - Number Art Coloring Book,4.3,967,2.8M,"100,000+",Everyone,Art & Design;Creativity,4.4 and up


In [5]:
apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
avgVer = pd.to_numeric(apps['Android Ver'], errors='coerce').mean()
apps['Android Ver'] = apps['Android Ver'].replace('', avgVer)
apps['Android Ver'] = apps['Android Ver'].str.replace('.','a', regex=True).str.replace('a','.', 1, regex=True).str.replace('a','', regex=True)
apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')

avgSize = pd.to_numeric(apps['Size'], errors='coerce').mean()
apps['Size'] = apps['Size'].replace('', avgSize)
apps = apps.dropna()
apps = apps.astype({'Reviews':'float', 'Size':'float', 'Installs':'float', 'Android Ver':'float'})

C:\Users\rafal\AppData\Local\Temp\ipykernel_18640\2251694235.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
C:\Users\rafal\AppData\Local\Temp\ipykernel_18640\2251694235.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
C:\Users\rafal\AppData\Local\Temp\ipykernel_18640\2251694235.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')


In [6]:
class DataClean:
    @staticmethod
    def splitGenres(x):
        genresConnected = x["Genres"].unique().tolist()
        genres = set(())
        for i in genresConnected:
            afterSplit = i.split(";")
            for j in afterSplit:
                if j!='February 11, 2018':
                    genres.add(j)
        for i in list(genres):
            x[i] = [0 for x in range(x.shape[0])]
        for index in x.index:
            values = x.loc[index,'Genres'].split(";")
            for i in values:
                x.loc[index,i] = 1
        return x

In [7]:
apps = DataClean.splitGenres(apps)
apps.head()
apps.drop(columns=['Genres'])

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Simulation,Photography,Lifestyle,...,Pretend Play,Action & Adventure,Music & Video,Puzzle,Role Playing,Art & Design,Events,Video Players & Editors,Card,Finance
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159.0,19.0,10000.0,Everyone,4.03,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,Coloring book moana,3.9,967.0,14.0,500000.0,Everyone,4.03,0,0,0,...,1,0,0,0,0,1,0,0,0,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510.0,8.7,5000000.0,Everyone,4.03,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,Sketch - Draw & Paint,4.5,215644.0,25.0,50000000.0,Teen,4.20,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,Pixel Draw - Number Art Coloring Book,4.3,967.0,2.8,100000.0,Everyone,4.40,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10832,FR Tides,3.8,1195.0,582.0,100000.0,Everyone,2.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10833,Chemin (fr),4.8,44.0,619.0,1000.0,Everyone,2.20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10834,FR Calculator,4.0,7.0,2.6,500.0,Everyone,4.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10836,Sya9a Maroc - FR,4.5,38.0,53.0,5000.0,Everyone,4.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames[:-1]:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row, column]-min1)/(max1-min1)
                x.at[row, column]=xprim
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [ ]:
DataProcessing.shuffle(apps)
apps = apps.reset_index(drop=True)

In [ ]:
DataProcessing.normalization(apps)

In [ ]:
apps.head()

In [ ]:
trainingSet, validatingSet = DataProcessing.split(apps, 0.999)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

In [ ]:
apps["Content Rating"].unique().tolist()

In [ ]:
if 'Mature 17+' in validatingSet["Content Rating"].unique().tolist() or 'Adults only 18+' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Adults only 18+' for i in range(len(validatingSet))]
elif 'Teen' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Teen' for i in range(len(validatingSet))]

In [ ]:
validatingSet.head()

<h2>k-nearest neighbors algorithm</h2>

In [ ]:
class KNN:
    @staticmethod
    def distance(v1, v2):
        sum = 0
        for j in range(1,len(v1)):
            if j != 5 and j!=6 and j!=7:
                sum += (v1[j] - v2[j])**2
        return sum**(1/2)
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNN.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] = 10
            if sample['Content Rating'] != 'Adults only 18+':
                if dataset.iloc[i]['Content Rating'] in ['Mature 17+','Adults only 18+']:
                    distances[i] = 10
            elif sample['Content Rating'] != 'Teen':
                if dataset.iloc[i]['Content Rating'] in ['Teen','Mature 17+','Adults only 18+']:
                    distances[i] = 10
        dataset["distance"] = distances
                    
        return dataset
    
    def recommendation(dataset,samples):
        sorted = []
        dataset["distance"] = [10 for x in range(len(dataset))]
        for index in range(len(samples)):
            sorted.append(KNN.clustering(dataset.copy(),samples.iloc[index]))
        distances = []
        for index in dataset.index:
#             dataset.loc[index,'distance']
            distance = []
            for i in range(len(sorted)):
                distance.append(sorted[i].loc[index,'distance'])
            avg = []
            for i in range(3):
                avg.append(min(distance))
                distance.remove(min(distance))
            distances.append(sum(avg)/3)
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        return dataset.head(10)

<h2>Apps Recommendation</h2>

<h4>Method 1</h4>

In [ ]:
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
maxAndroidVer = max(validatingSet['Android Ver'])
genres = []
for i in validatingSet.columns.tolist()[8:]:
    genres.append(pd.to_numeric(validatingSet[i], errors='coerce').mean())
avgAppList = ['avgApp',avgRating,avgReviews,avgSize,avgInstalls,validatingSet.iloc[0]['Content Rating'],"avgGenre",maxAndroidVer]
for i in genres:
    avgAppList.append(i)
avgApp = pd.DataFrame(columns=validatingSet.columns.tolist())
avgApp.loc[0] = avgAppList
recommendedApps = KNN.clustering(trainingSet.copy(), validatingSet.iloc[0])
recommendedApps.sort_values("distance").head()

In [ ]:
validatingSet.head()

<h4>Method 2</h4>

In [ ]:
KNN.recommendation(trainingSet.copy(),validatingSet.copy())

<h2>Importing Reviews Database</h2>

In [ ]:
reviews = pd.read_csv(r"processedReviews.csv")

In [ ]:
display(reviews)

In [ ]:
appsWithReviews = pd.merge(apps, reviews, on="App")

In [ ]:
display(appsWithReviews)

In [ ]:
trainingSet, validatingSet = DataProcessing.split(appsWithReviews, 0.995)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

In [ ]:
if 'Mature 17+' in validatingSet["Content Rating"].unique().tolist() or 'Adults only 18+' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Adults only 18+' for i in range(len(validatingSet))]
elif 'Teen' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Teen' for i in range(len(validatingSet))]

In [ ]:
class KNNWithJaccardSimilarityCoefficient:
    @staticmethod
    def distance(v1, v2):
        sum = 0
        for j in range(1,len(v1)-1):
            if j != 5 and j!=6 and j!=7:
                sum += (v1[j] - v2[j])**2
        return (sum**(1/2)) * KNNWithJaccardSimilarityCoefficient.jaccard(v1[len(v1)-1], v2[len(v1)-1])
    
    def jaccard(string1,string2):
        sim = 0
        try:
            string1 = string1.split(";")
        except:
            string1 = [""]       
        try:
            string2 = string2.split(";")
        except:
            string2 = [""]
            
        for word in string1:
            if word in string2:
                sim+=1
        return 1 - (sim/5)
    
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNNWithJaccardSimilarityCoefficient.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] = 10
            if sample['Content Rating'] != 'Adults only 18+':
                if dataset.iloc[i]['Content Rating'] in ['Mature 17+','Adults only 18+']:
                    distances[i] = 10
            elif sample['Content Rating'] != 'Teen':
                if dataset.iloc[i]['Content Rating'] in ['Teen','Mature 17+','Adults only 18+']:
                    distances[i] = 10
        dataset["distance"] = distances
                    
        return dataset
    
    def recommendation(dataset,samples):
        sorted = []
        dataset["distance"] = [10 for x in range(len(dataset))]
        for index in range(len(samples)):
            sorted.append(KNNWithJaccardSimilarityCoefficient.clustering(dataset.copy(),samples.iloc[index]))
        distances = []
        for index in dataset.index:
#             dataset.loc[index,'distance']
            distance = []
            for i in range(len(sorted)):
                distance.append(sorted[i].loc[index,'distance'])
            avg = []
            for i in range(3):
                avg.append(min(distance))
                distance.remove(min(distance))
            distances.append(sum(avg)/3)
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        return dataset.head(10)

<h4>Method 1</h4>

In [ ]:
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
maxAndroidVer = max(validatingSet['Android Ver'])
genres = []
for i in validatingSet.columns.tolist()[8:-1]:
    genres.append(pd.to_numeric(validatingSet[i], errors='coerce').mean())
avgAppList = ['avgApp',avgRating,avgReviews,avgSize,avgInstalls,validatingSet.iloc[0]['Content Rating'],"avgGenre",maxAndroidVer]
for i in genres:
    avgAppList.append(i)
revs = []
for i in validatingSet["reviews"].tolist():
    i = i.split(";")
    for j in i:
        revs.append(j)
words = {}
for word in revs:
        words[word] = 0
for word in revs:
        words[word] +=1
revs = []
for i in range(5):
    try:
        revs.append(max(words, key=words.get))
        del words[max(words, key=words.get)]
    except:
        pass
avgWords = ";".join(revs)
avgAppList.append(avgWords)

avgApp = pd.DataFrame(columns=validatingSet.columns.tolist())
avgApp.loc[0] = avgAppList
recommendedApps = KNNWithJaccardSimilarityCoefficient.clustering(trainingSet.copy(), validatingSet.iloc[0])
recommendedApps.sort_values("distance").head(10)

<h4>Method 2</h4>

In [ ]:
KNNWithJaccardSimilarityCoefficient.recommendation(trainingSet.copy(),validatingSet.copy())